<a href="https://colab.research.google.com/github/ilopezro/cse143/blob/jen-assg2/NEW_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [0]:
tf.random.set_seed(42)

In [1]:
import tensorflow_datasets as tfds

#splitting of training and dev data
train_data, dev_data, test_data = tfds.load("imdb_reviews", split=('train[:80%]', 'train[80%:]', 'test'), as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteY42BFL/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteY42BFL/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteY42BFL/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [0]:
from collections import Counter

# creating a vocabulary from training data 
vocabulary = Counter()
for X_batch, y_batch in train_data.batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [0]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [0]:
word_to_id_train = {word: index for index, word in enumerate(truncated_vocabulary)}

In [0]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [0]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = train_data.batch(256).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)
dev_set = dev_data.batch(256).map(preprocess)
dev_set = dev_set.map(encode_words).prefetch(1)


In [10]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)
for X_batch, y_batch in dev_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[  22   11   28 ...    0    0    0]
 [   6   21   73 ...    0    0    0]
 [3453 6662    1 ...    0    0    0]
 ...
 [  14   62  288 ...    0    0    0]
 [7854 8574  147 ...    0    0    0]
 [ 826  234  100 ...    0    0    0]], shape=(256, 63), dtype=int64)
tf.Tensor(
[0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1
 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 0 1 0 0 0 1 0
 1 1 1 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 1 1 1 0 1 1 0 1 0 0 1 0 1 1 0 0
 1 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1
 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 0 1 1 1 0 0 0
 1 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 1 0 1 0
 0 1 0 0 1 1 0 0 0 1 0 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 0 1], shape=(256,), dtype=int64)
tf.Tensor(
[[ 1273   161     5 ...     0     0     0]
 [  277    11  1668 ...     0     0     0]
 [    6   269 10627 ...     0     0     0]
 ...
 [ 2574    63  3005 ...   

In [11]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.SimpleRNN(128, activation="elu", dropout=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(train_set, epochs=20, validation_data=dev_set)

Epoch 1/20
79/79 [==============================] - 11s 141ms/step - loss: 0.6798 - accuracy: 0.5659 - val_loss: 0.6235 - val_accuracy: 0.6586
Epoch 2/20
79/79 [==============================] - 9s 108ms/step - loss: 0.5070 - accuracy: 0.7728 - val_loss: 0.6942 - val_accuracy: 0.7124
Epoch 3/20
79/79 [==============================] - 9s 109ms/step - loss: 0.3213 - accuracy: 0.8695 - val_loss: 0.5752 - val_accuracy: 0.7460
Epoch 4/20
79/79 [==============================] - 9s 111ms/step - loss: 0.1961 - accuracy: 0.9255 - val_loss: 0.7015 - val_accuracy: 0.6806
Epoch 5/20
79/79 [==============================] - 9s 109ms/step - loss: 0.1019 - accuracy: 0.9651 - val_loss: 1.0632 - val_accuracy: 0.5894
Epoch 6/20
79/79 [==============================] - 9s 111ms/step - loss: 0.0658 - accuracy: 0.9789 - val_loss: 0.8011 - val_accuracy: 0.7180
Epoch 7/20
79/79 [==============================] - 9s 111ms/step - loss: 0.0341 - accuracy: 0.9894 - val_loss: 0.9258 - val_accuracy: 0.7090
Epoch